### comment out the code below to install required packages

In [1]:
# code to install all the packages we need
# %pip install numpy
# %pip install pandas
# %pip install yfinance
# %pip install requests
# %pip install beautifulsoup4
# %pip install scikit-learn
# %pip install TA-lib
# %pip install joblib
# %pip install pickle
# %pip install Riskfolio-lib


In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
import yfinance as yf
from ibapi.client import *
from ibapi.wrapper import *
from ibapi.contract import *
from ibapi.order import *
import ibapi

import requests
import bs4 as bs
import datetime
import time
import threading
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV  
import joblib

import talib as ta
from talib import MA_Type
import pickle


In [3]:
init_event = threading.Event() # init event
id_event = threading.Event()
mkt_event = threading.Event()
hist_event = threading.Event()
order_event = threading.Event()
evec_event = threading.Event()
port_event = threading.Event()
value_event = threading.Event()

In [4]:
# define the App class
class App(EClient, EWrapper):
    def __init__(self, address, port, cid):
        EClient.__init__(self, self)
        # # list to store data
        self.bar_dict = {}
        self.mkt_price = []
        # create a connection with IBKR
        self.connect(address, port, cid)
        self.last_portfolio = pd.DataFrame(columns=["ticker","quantity","marketPrice","marketValue","averageCost","unrealizedPNL","realizedPNL"])
        self.value = 0
        # start client
        thread = threading.Thread(target=self.run)
        thread.start()
        init_event.set()


    def nextValidId(self, orderId: int):
        # provide a new order id for each of my requests
        super().nextValidId(orderId)
        logging.debug("setting nextValidOrderId: %d", orderId)
        self.nextValidOrderId = orderId
        print("NextValidId:", orderId)
        id_event.set()

    def tickPrice(self, reqId: int, tickType: int, price: float, attrib: ibapi.common.TickAttrib):
            print("Tick Price. Ticker Id:", reqId, "tickType:", tickType, "Price:", price)
            if tickType == 9: # if tickType is Close Price
                self.mkt_price.append([reqId, price])
                mkt_event.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.bar_dict.keys():
            self.bar_dict[reqId] = []
        self.bar_dict[reqId].append(vars(bar))
        
    def historicalDataEnd(self, reqId, start, end):
        print(f"end of historicalData")
        hist_event.set()


    # implement code to monitor trade status and receive confirmation of the trade
    def openOrder(self, orderId: OrderId, contract: Contract, order: Order, orderstate: OrderState):
        # openorder callback
        print(f"openOrder. orderId:{orderId}, contract:{contract}, order:{order}")
        order_event.set()

    def orderStatus(self, orderId: OrderId, status: str, filled: float, reamining: float, avgFillPrice: float,
                    permId: int, parenId: int, lastFillPrice: float, clientId: int, whyHeld:str, mktCapPrice: float):
                    # orderstatus callback
        print(f"orderStatus. orderId: {orderId}, status: {status}, filled: {filled}, remaining:{reamining}, avgFillPrice: {avgFillPrice}, permId:{permId}, parentId:{parenId}, lastFillPrice: {lastFillPrice}, clientId: {clientId}, whyHeld: {whyHeld}, mktCapPrice:{mktCapPrice}")

    def execDetails(self, reqId: int, contract: Contract, execution: Execution):
        print(f"execDetails. reqId: {reqId}, contract: {contract}, execution: {execution}")
        evec_event.set()
    # basically a summary

    def commissionReport(self, commissionReport: CommissionReport):
        super().commissionReport(commissionReport)
        print("CommissionReport.", commissionReport)
        
    # called when query portfolio information
    def updatePortfolio(self, contract, position, marketPrice, marketValue, averageCost, unrealizedPNL, realizedPNL, accountName):
        self.last_portfolio = pd.concat([self.last_portfolio,
                                        pd.DataFrame([[contract.symbol, position, marketPrice, marketValue, averageCost, unrealizedPNL, realizedPNL]],
                                                    columns=self.last_portfolio.columns)],
                                        ignore_index=True)
        port_event.set()

    # query total value
    def accountSummary(self, reqId: int, account: str, tag: str, value: str, currency: str):
        self.value = float(value)
        print("Total value of the account: ", self.value)
    
    def accountSummaryEnd(self, reqId: int):
        print('end of account summary')
        value_event.set()

In [5]:
# Connect to the TWS API
app = App('127.0.0.1', 7497, 1000)
init_event.wait() # wait until it's connected
init_event.clear()

NextValidId: 44


ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:eufarmnj
ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR:ibapi.wrapper:ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR:ibapi.wrapper:ERROR -1 2106 HMDS data farm connection is OK:ushmds.nj
ERROR:ibapi.wrapper:ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR:ibapi.wrapper:ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR:ibapi.wrapper:ERROR -1 2158 Sec-def data farm connection is OK:secdefil


In [6]:
app.reqMarketDataType(1) # in case it doesn't work, change 1 to 3

In [7]:
with open('portfolio_list.pickle', 'rb') as f:
    portfolio_list = pickle.load(f)

print(portfolio_list)

['A', 'AWK', 'AZO', 'BXP', 'IFF', 'MRO', 'MHK', 'MOH', 'PKI', 'VFC']


# query the current portfolio


In [8]:
app.reqAccountUpdates(True, "DU6730183") #TODO: replace the DU222526 with our account ID
# port_event.wait()
# port_event.clear()

time.sleep(10)
last_portfolio = app.last_portfolio
app.reqAccountUpdates(False, "DU6730183") 

ERROR:ibapi.wrapper:ERROR -1 2100 API client has been unsubscribed from account data.


In [9]:
last_portfolio

,ticker,quantity,marketPrice,marketValue,averageCost,unrealizedPNL,realizedPNL
0,ADP,385,221.816528,85399.36,221.340649,183.21,0.0
1,AJG,458,188.793991,86467.65,184.939814,1765.21,0.0
2,DVN,1539,54.026112,83146.19,54.786397,-1170.08,0.0
3,FITB,2407,32.323902,77803.63,35.807464,-8384.93,0.0
4,KMX,1263,69.119652,87298.12,68.351374,970.34,0.0
5,LNC,2795,28.283646,79052.79,31.680190,-9493.34,0.0
6,LYV,1183,69.992950,82801.66,75.391826,-6386.87,0.0
7,TRGP,1110,76.295578,84688.09,76.402778,-118.99,0.0
8,YUM,666,127.669273,85027.74,128.625848,-637.08,0.0
9,ZBH,697,123.575371,86132.03,122.517252,737.51,0.0


In [10]:
app.reqAccountUpdates(False, "DU6730183")#TODO: replace the DU222526 with our account ID
app.reqAccountSummary(9001, "All", "NetLiquidation")
value_event.wait()
value_event.clear()
# time.sleep(3)
total_value = app.value
app.cancelAccountSummary(9001)

Total value of the account:  924869.84
end of account summary


In [11]:
total_value

924869.84

# implementation
* decide what to buy & sell, as well as the quantity

In [12]:

# get the market price
for i, t in enumerate(portfolio_list):
    contract = Contract()
    contract.symbol = t
    contract.secType = "STK"
    contract.currency = "USD"
    contract.exchange = "SMART"
    app.reqMktData(i, contract, "", False, False, [])
    mkt_event.wait()
    mkt_event.clear()
    app.cancelMktData(i)

mkt_df = pd.DataFrame(app.mkt_price, columns=['reqId', 'mkt_price'])
assert sum(mkt_df.isnull().any()) == 0, "didn't get all the market price"
assert len(mkt_df) == len(portfolio_list), "didn't get all the market price"


ERROR:ibapi.wrapper:ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.hfarm
ERROR:ibapi.wrapper:ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.hfarm
ERROR:ibapi.wrapper:ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.jfarm
ERROR:ibapi.wrapper:ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.jfarm
ERROR:ibapi.wrapper:ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.usfuture
ERROR:ibapi.wrapper:ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.usfuture
ERROR:ibapi.wrapper:ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.eufarmnj
ERROR:ibapi.wrapper:ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.eufarmnj


Tick Price. Ticker Id: 0 tickType: 6 Price: 142.0
Tick Price. Ticker Id: 0 tickType: 7 Price: 140.91
Tick Price. Ticker Id: 0 tickType: 9 Price: 140.9
Tick Price. Ticker Id: 0 tickType: 4 Price: 141.27
Tick Price. Ticker Id: 0 tickType: 14 Price: 141.6
Tick Price. Ticker Id: 0 tickType: 1 Price: 141.18
Tick Price. Ticker Id: 0 tickType: 2 Price: 141.35
Tick Price. Ticker Id: 1 tickType: 14 Price: 138.11
Tick Price. Ticker Id: 1 tickType: 6 Price: 138.52
Tick Price. Ticker Id: 1 tickType: 7 Price: 136.95
Tick Price. Ticker Id: 1 tickType: 9 Price: 138.24
Tick Price. Ticker Id: 1 tickType: 4 Price: 138.04
Tick Price. Ticker Id: 1 tickType: 1 Price: 138.07
Tick Price. Ticker Id: 1 tickType: 2 Price: 138.18
Tick Price. Ticker Id: 2 tickType: 14 Price: 2453.95
Tick Price. Ticker Id: 2 tickType: 4 Price: 2439.47
Tick Price. Ticker Id: 2 tickType: 6 Price: 2467.1
Tick Price. Ticker Id: 2 tickType: 7 Price: 2422.16
Tick Price. Ticker Id: 2 tickType: 9 Price: 2459.57
Tick Price. Ticker Id: 2 ti

In [13]:
target_portfolio = pd.DataFrame(columns=['tickers', 'weight', 'mkt_price'])
target_portfolio['tickers'] = portfolio_list
target_portfolio['weight'] = np.repeat(1/len(portfolio_list), len(portfolio_list))
target_portfolio['mkt_price'] = mkt_df['mkt_price']

In [14]:
target_portfolio['quantity'] = np.floor(target_portfolio['weight'] * (total_value * 0.95) / target_portfolio['mkt_price'])

In [15]:
last_portfolio

,ticker,quantity,marketPrice,marketValue,averageCost,unrealizedPNL,realizedPNL
0,ADP,385,221.816528,85399.36,221.340649,183.21,0.0
1,AJG,458,188.793991,86467.65,184.939814,1765.21,0.0
2,DVN,1539,54.026112,83146.19,54.786397,-1170.08,0.0
3,FITB,2407,32.323902,77803.63,35.807464,-8384.93,0.0
4,KMX,1263,69.119652,87298.12,68.351374,970.34,0.0
5,LNC,2795,28.283646,79052.79,31.680190,-9493.34,0.0
6,LYV,1183,69.992950,82801.66,75.391826,-6386.87,0.0
7,TRGP,1110,76.295578,84688.09,76.402778,-118.99,0.0
8,YUM,666,127.669273,85027.74,128.625848,-637.08,0.0
9,ZBH,697,123.575371,86132.03,122.517252,737.51,0.0


In [16]:
target_portfolio

,tickers,weight,mkt_price,quantity
0,A,0.1,140.90,623.0
1,AWK,0.1,138.24,635.0
2,AZO,0.1,2459.57,35.0
3,BXP,0.1,64.17,1369.0
4,IFF,0.1,87.99,998.0
5,MRO,0.1,25.39,3460.0
6,MHK,0.1,101.55,865.0
7,MOH,0.1,269.48,326.0
8,PKI,0.1,124.18,707.0
9,VFC,0.1,23.72,3704.0


# figure out the best order of our order. (To save commission fee)

* target_portfolio
> target_portfolio python pandas dataframe containing information of stocks portfolio(target portfolio) that I want to form through trading. The first column of the dataframe is stocks' tickers and the second column of the dataframe is the quantity of each stock that I want to hold. 

* last_portfolio
> last_portfolio is a dataframe containing information of last portfolio that I hold

In [17]:
# Create a list to store the stocks that need to be sold
stocks_to_sell = []
# Create a list to store the stocks that need to buy
stocks_to_buy = []

# Iterate through the target_portfolio dataframe
for index, row in target_portfolio.iterrows():
    # Check if the stock is present in the last_portfolio dataframe
    last_portfolio_stock = last_portfolio[last_portfolio['ticker'] == row['tickers']]
    if not last_portfolio_stock.empty:
        # Compare the quantities
        last_portfolio_quantity = float(last_portfolio_stock['quantity'].values[0])
        if row['quantity'] < last_portfolio_quantity:
            # Calculate the difference in quantity
            quantity_diff = last_portfolio_quantity - row['quantity'] 
            # Add the stock to the list of stocks to sell
            stocks_to_sell.append([row['tickers'], quantity_diff])

        if row['quantity'] > last_portfolio_quantity:
            # Calculate the difference in quantity
            quantity_diff = row['quantity'] - last_portfolio_quantity 
            # Add the stock to the list of stocks to buy
            stocks_to_buy.append([row['tickers'], quantity_diff])

# we also need to sell the stocks that are in last_portfolio but not in target_portfolio
# Find the difference between the two portfolios
new_tickers_to_sell = set(last_portfolio['ticker']).difference(set(target_portfolio['tickers']))

for ticker in new_tickers_to_sell:
    # Find the quantity of the stock to sell from the last_portfolio
    quantity = last_portfolio.loc[last_portfolio['ticker'] == ticker, 'quantity'].values[0]
    # append to stocks to sell
    if quantity != 0:
        stocks_to_sell.append([ticker, quantity])

# we also need to buy the stocks that are in target_protfolio but not in last_portfolio
# Find the difference between the two portfolios
new_tickers_to_buy = set(target_portfolio['tickers']).difference(set(last_portfolio['ticker']))

for ticker in new_tickers_to_buy:
    # Find the quantity of the stock to buy from the target_portfolio
    quantity = target_portfolio.loc[target_portfolio['tickers'] == ticker, 'quantity'].values[0]
    # append to stocks to buy
    if quantity != 0:
        stocks_to_buy.append([ticker, quantity])

In [18]:
stocks_to_buy

[['AZO', 35.0],
 ['MOH', 326.0],
 ['MRO', 3460.0],
 ['A', 623.0],
 ['MHK', 865.0],
 ['AWK', 635.0],
 ['BXP', 1369.0],
 ['VFC', 3704.0],
 ['IFF', 998.0],
 ['PKI', 707.0]]

In [19]:
stocks_to_sell

[['ADP', Decimal('385')],
 ['ZBH', Decimal('697')],
 ['DVN', Decimal('1539')],
 ['FITB', Decimal('2407')],
 ['YUM', Decimal('666')],
 ['KMX', Decimal('1263')],
 ['TRGP', Decimal('1110')],
 ['LYV', Decimal('1183')],
 ['LNC', Decimal('2795')],
 ['AJG', Decimal('458')]]

# place order

In [20]:
app.reqIds(-1) # require a new id
id_event.wait()
id_event.clear()
orderid = app.nextValidOrderId
for stock in stocks_to_sell:
    # Replace the place holder values with the appropriate values
    ticker = stock[0]
    quantity = stock[1]
    contract = Contract()
    contract.symbol = ticker
    contract.secType = "STK"
    contract.exchange = "SMART"
    contract.currency = "USD"
    order = Order()
    order.action = "SELL"
    order.orderType = "MKT"
    order.totalQuantity = quantity
    order.tif = "GTC" # default is day order

    orderid += 1

    app.placeOrder(orderid, contract, order)
    order_event.wait()
    order_event.clear()

count = 0
for stock in stocks_to_buy:
    # Replace the place holder values with the appropriate values
    app.reqAccountUpdates(False, "DU6730183")#TODO: replace the DU222526 with our account ID
    app.reqAccountSummary(9001, "All", "SettledCash")
    value_event.wait()
    value_event.clear()
    cash_value = app.value
    if count >= len(stocks_to_buy)/2 and cash_value <= 0:
        break # stop buying when cash is below 0
    ticker = stock[0]
    quantity = stock[1]
    contract = Contract()
    contract.symbol = ticker
    contract.secType = "STK"
    contract.exchange = "SMART"
    contract.currency = "USD"
    order = Order()
    order.action = "BUY"
    order.orderType = "MKT"
    order.totalQuantity = quantity
    order.tif = "GTC" # default is day order

    orderid += 1
    app.placeOrder(orderid, contract, order)
    order_event.wait()
    order_event.clear()
    count +=1

NextValidId: 44
openOrder. orderId:45, contract:4661,ADP,STK,,0,?,,SMART,,USD,ADP,NMS,False,,,,combo:, order:45,1000,1182131027: MKT SELL 385@221.65 GTC
orderStatus. orderId: 45, status: PreSubmitted, filled: 0, remaining:385, avgFillPrice: 0.0, permId:1182131027, parentId:0, lastFillPrice: 0.0, clientId: 1000, whyHeld: , mktCapPrice:0.0
execDetails. reqId: -1, contract: 4661,ADP,STK,,0,,,ISLAND,,USD,ADP,NMS,False,,,,combo:, execution: ExecId: 0000e0d5.643926d7.01.01, Time: 20230309 12:54:20 US/Eastern, Account: DU6730183, Exchange: ISLAND, Side: SLD, Shares: 100, Price: 221.69, PermId: 1182131027, ClientId: 1000, OrderId: 45, Liquidation: 0, CumQty: 100, AvgPrice: 221.69, OrderRef: , EvRule: , EvMultiplier: 0, ModelCode: , LastLiquidity: 2
openOrder. orderId:45, contract:4661,ADP,STK,,0,?,,SMART,,USD,ADP,NMS,False,,,,combo:, order:45,1000,1182131027: MKT SELL 385@221.65 GTC
orderStatus. orderId: 45, status: PreSubmitted, filled: 100, remaining:285, avgFillPrice: 221.69, permId:1182131

In [21]:
app.disconnect()

execDetails. reqId: -1, contract: 9831,MRO,STK,,0,,,IEX,,USD,MRO,MRO,False,,,,combo:, execution: ExecId: 00025b45.6409f3d0.01.01, Time: 20230309 12:54:22 US/Eastern, Account: DU6730183, Exchange: IEX, Side: BOT, Shares: 146, Price: 24.91, PermId: 1182131039, ClientId: 1000, OrderId: 57, Liquidation: 0, CumQty: 3358, AvgPrice: 24.91, OrderRef: , EvRule: , EvMultiplier: 0, ModelCode: , LastLiquidity: 2
